# Predicting winners 



In [9]:
import pandas as pd
import numpy as np
import logging
import json
from datetime import datetime
from sklearn.utils import resample
from sklearn.model_selection import train_test_split
import sklearn


pd.options.display.max_columns=None

We can read in the nominee reviews. These are all reviews for all nominees. Some end up winning, others do not. 

In [2]:
awardReviews=pd.read_csv('awardReviews.csv')
print(len(awardReviews))
print(awardReviews.columns)

320535
Index(['bookID', 'bookLink', 'bookTitle', 'authorName', 'award', 'loser',
       'winner', 'user_id', 'book_id', 'review_id', 'rating', 'review_text',
       'date_added', 'date_updated', 'read_at', 'started_at', 'n_votes',
       'n_comments'],
      dtype='object')


In [3]:
awardReviews.winner.value_counts()

0    212341
1    108194
Name: winner, dtype: int64

About half of our nominee reviews are for winners. 

In [5]:
awardReviews['reviewDate'] = pd.to_datetime(awardReviews.date_added,utc=True)

## Plan 

We need to train a NLP model to classify the reviews. From there we will predict for each review whether we think it is for a winner or loser

1. Transformer model
2. aggregate
3. traditional classifier

In [39]:
from simpletransformers.classification import ClassificationModel

awardTexts = awardReviews[['bookID','review_text','winner','n_votes']].rename(columns={'review_text':'text','winner':'labels'})

awardTexts['text'] = awardTexts['text'].astype(str).str.replace('\D+', '')

In [13]:
#X = awardTexts.drop('labels',axis=1)
#y = awardTexts.labels

#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

#train = pd.concat([X_train, y_train], axis=1)
#test = pd.concat([X_test,y_test],axis=1)
#print('{} in train set, {} in test'.format(len(train),len(test)))


214758 in train set, 105777 in test


In [44]:
#train, validate, test = np.split(awardTexts.sample(frac=1), [int(.6*len(awardTexts)), int(.8*len(awardTexts))])
#print('{} in train, {} in validate, {} in test'.format(len(train),len(validate),len(test)))
train, test = np.split(awardTexts.sample(frac=1), [int(.8*len(awardTexts))])

In [47]:
from sklearn.utils import resample

# separate minority and majority classes
lost = train[train.labels==0]
won = train[train.labels==1]

# upsample minority
win_resampled = resample(won,
                          replace=True, # sample with replacement
                          n_samples=len(lost), # match number in majority class
                          random_state=42) # reproducible results

# combine majority and upsampled minority
upsampled = pd.concat([lost,win_resampled])

# check new class counts
upsampled.labels.value_counts()

1    170023
0    170023
Name: labels, dtype: int64

In [49]:
train,validate = np.split(upsampled.sample(frac=1), [int(0.7*len(upsampled))])
print('{} train, {} validate, {} final test'.format(len(train),len(validate),len(test)))

238032 train, 102014 validate, 64107 final test


Let's sample from these to make it runnable.

In [50]:
train_small = train.sample(n=100000)[['text','labels']]
eval_df = validate.sample(n=10000)[['text','labels']]

In [35]:
test[['text']].dtypes

text    object
dtype: object

In [51]:
# Optional model configuration
model_args = {
    "num_train_epochs": 5,
    "sliding_window": True,
    'use_early_stopping': True,
    'early_stopping_patience': 3,
    "early_stopping_delta": 0.01,
    "early_stopping_metric": "mcc",
    "evaluate_during_training":True,
    "evaluate_during_training_steps": 1000,
    "save_model_every_epoch": False,
    'overwrite_output_dir': True,
    'no_cache': True,
    'output_dir':'transformers-outputs/winners',
    #'weight':[0.5]
}

# Create a ClassificationModel
model = ClassificationModel(
    "electra", "google/electra-small-discriminator", args=model_args
)

# Train the model
model.train_model(train_small,eval_df=eval_df)


Selected optimization level O1:  Insert automatic casts around Pytorch functions and Tensor methods.

Defaults for this optimization level are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic
Processing user overrides (additional kwargs that are not None)...
After processing overrides, optimization options are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic


Running loss: 0.783035Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 32768.0
Running loss: 0.588914Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 16384.0
Running loss: 0.771424Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 8192.0
Running loss: 0.691651



In [52]:
#test['text'] = test['text'].astype(str).str.replace('\D+', '')

In [53]:

result, model_outputs, wrong_predictions = model.eval_model(test[['text','labels']],acc=sklearn.metrics.accuracy_score,f1=sklearn.metrics.f1_score)
print(result)


{'mcc': 0.0, 'tp': 0, 'tn': 42318, 'fp': 0, 'fn': 21789, 'acc': 0.6601151200336937, 'f1': 0.0, 'eval_loss': 0.6899995799891099}
